# Chapter 10: Bet Sizing Mechanics
AFML focus: convert conviction and divergence into bounded bet sizes.


In [1]:
import math
import matplotlib.pyplot as plt
import polars as pl
import openquant
import sys
from pathlib import Path
sys.path.insert(0, str(Path('notebooks/python/scripts').resolve()))
from afml_chapter_utils import (
    fetch_panel,
    simple_returns,
    probs_and_sides_from_momentum,
    timestamps_from_dates,
    lag_corr,
    fracdiff_ffd,
)

panel = fetch_panel(window=900)
dates = panel['date'].to_list()
uso = panel['USO'].to_list()
uso_ret = simple_returns(uso)
probs, sides = probs_and_sides_from_momentum(uso)
timestamps = timestamps_from_dates(dates)
asset_names = ['USO', 'BNO', 'XLE', 'GLD', 'UNG']
asset_prices = panel.select(asset_names).rows()
print('rows', panel.height, 'range', dates[0], dates[-1])


rows 900 range 2022-07-08 2026-02-06


In [2]:
div = [x/100 for x in range(-80, 81, 2)]
sigmoid = [openquant.bet_sizing.bet_size(10.0, d, 'sigmoid') for d in div]
power = [openquant.bet_sizing.bet_size(2.0, max(min(d,1.0),-1.0), 'power') for d in div]

plt.figure(figsize=(8,4))
plt.plot(div, sigmoid, label='sigmoid')
plt.plot(div, power, label='power')
plt.title('Chapter 10: Bet Size Functions')
plt.legend()
plt.tight_layout()
plt.show()

print('sigmoid mid', openquant.bet_sizing.bet_size(10.0, 0.1, 'sigmoid'))


sigmoid mid 0.0316069770620507


## Interpretation
The mapping enforces bounded exposures and smooth position response to forecast divergence.

